In [7]:
import numpy as np
import pandas as pd
import time
import joblib  # For loading the trained model
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Load preprocessed datasets
attack_free = pd.read_csv('extracted feature/CAN_attack_dataset1/Attack_free new.csv')[0:500]
dos = pd.read_csv('extracted feature/CAN_attack_dataset1/DoS_Attack_new.csv')[0:200]
fuzzy = pd.read_csv('extracted feature/CAN_attack_dataset1/Fuzzy_Attack_New.csv')[0:300]
impersonation = pd.read_csv('extracted feature/CAN_attack_dataset1/Impersonation_Attack_New.csv')[0:400]

def preprocess_data(df):
    label_encoder = LabelEncoder()
    # Process all columns except (optionally) the first if it is not categorical
    for col in df.columns[1:]:
        df[col] = label_encoder.fit_transform(df[col])
    return df

# Apply preprocessing
attack_free = preprocess_data(attack_free)
dos = preprocess_data(dos)
fuzzy = preprocess_data(fuzzy)
impersonation = preprocess_data(impersonation)

# Combine datasets
datasets = [(attack_free, 1), (dos, 2), (fuzzy, 3), (impersonation, 4)]

# Feature Scaling: **Include all columns**
dataset_combined = np.vstack([df.values for df, _ in datasets])
scaler = StandardScaler()
scaler.fit(dataset_combined)

# Load the trained SVM model from the .pkl file
model = joblib.load("svm_model.pkl")  # This model expects 16 features

def send_data_real_time(df, label, duration=15):
    for i in range(0, len(df), 15):  # Send 15 rows at a time
        if i + 15 > len(df):
            break  # Stop if there are fewer than 15 rows left
        batch = df.iloc[i:i+15, :].values  # Use all columns (16 features)
        batch = scaler.transform(batch)     # Apply scaling
        
        # Check if the number of features matches the model's expected number
        if batch.shape[1] != model.n_features_in_:
            print(f"Warning: Expected {model.n_features_in_} features, but got {batch.shape[1]} features.")
            continue  # Skip this batch if the number of features is incorrect
        
        predictions = model.predict(batch)  # Use the loaded model
        print(f"[{label}] Predictions: {predictions}")
        time.sleep(1)  # Wait for 1 second

# Run testing for each dataset sequentially
for df, label in datasets:
    send_data_real_time(df, label)


[1] Predictions: [2 1 1 4 1 1 1 1 4 1 1 4 4 1 1]
[1] Predictions: [4 4 1 1 1 1 4 1 1 4 1 1 1 1 1]
[1] Predictions: [1 1 4 1 1 4 1 1 1 4 4 1 4 1 1]
[1] Predictions: [1 4 4 1 1 1 1 3 1 1 4 1 1 1 1]
[1] Predictions: [1 1 4 1 4 4 1 1 4 1 1 4 3 1 1]
[1] Predictions: [1 1 1 4 1 1 1 2 4 4 1 1 3 1 4]
[1] Predictions: [1 1 1 3 1 1 4 1 1 4 1 1 4 4 1]
[1] Predictions: [4 4 1 3 1 1 1 1 1 1 1 1 4 1 1]
[1] Predictions: [1 2 1 1 1 1 4 1 1 1 1 4 1 1 1]
[1] Predictions: [1 4 4 4 2 1 1 2 1 1 1 1 1 1 4]
[1] Predictions: [1 1 1 4 1 4 1 1 4 1 1 1 2 1 1]
[1] Predictions: [1 1 4 1 1 1 1 1 1 1 1 1 4 1 2]
[1] Predictions: [1 1 4 1 1 4 1 1 4 1 1 4 1 1 4]
[1] Predictions: [2 1 1 4 1 1 2 1 1 1 4 1 4 1 4]
[1] Predictions: [1 1 1 4 1 4 1 1 1 4 4 4 1 1 4]
[1] Predictions: [1 4 1 1 1 1 1 1 4 1 1 1 1 1 1]
[1] Predictions: [1 1 4 4 1 4 1 1 1 1 1 1 1 1 1]
[1] Predictions: [1 1 1 1 4 4 4 1 1 1 1 4 4 1 1]
[1] Predictions: [1 1 1 1 1 1 1 1 1 1 1 4 1 1 1]
[1] Predictions: [1 1 4 1 1 1 1 1 1 1 1 1 1 1 1]
[1] Predictions: [1 

In [8]:
import numpy as np
import pandas as pd
import time
import joblib  # For loading the trained model
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Load preprocessed datasets (example with limited rows)
attack_free = pd.read_csv('extracted feature/CAN_attack_dataset1/Attack_free new.csv')[0:500]
dos = pd.read_csv('extracted feature/CAN_attack_dataset1/DoS_Attack_new.csv')[0:200]
fuzzy = pd.read_csv('extracted feature/CAN_attack_dataset1/Fuzzy_Attack_New.csv')[0:300]
impersonation = pd.read_csv('extracted feature/CAN_attack_dataset1/Impersonation_Attack_New.csv')[0:400]

def preprocess_data(df):
    label_encoder = LabelEncoder()
    for col in df.columns[1:]:  # Assuming first column is timestamp or ID
        df[col] = label_encoder.fit_transform(df[col])
    return df

# Apply preprocessing
attack_free = preprocess_data(attack_free)
dos = preprocess_data(dos)
fuzzy = preprocess_data(fuzzy)
impersonation = preprocess_data(impersonation)

# Combine datasets with a dataset label for reference
datasets = [(attack_free, 1), (dos, 2), (fuzzy, 3), (impersonation, 4)]

# Feature Scaling
scaler = StandardScaler()
# Note: Adjust here if you want to include or exclude the first column. 
# For example, if the first column is not a feature, use df.iloc[:, 1:]
dataset_combined = np.vstack([df.values for df, _ in datasets])
scaler.fit(dataset_combined)

# Load the trained SVM model from the .pkl file
model = joblib.load("svm_model.pkl")  # The model should expect the same number of features as scaler was fit on

# Mapping from numeric prediction to attack type
attack_mapping = {
    1: "Attack Free",
    2: "DoS",
    3: "Fuzzy",
    4: "Impersonation"
}

def get_majority_label(predictions):
    """
    Given an array of predictions, determine the majority label.
    """
    unique, counts = np.unique(predictions, return_counts=True)
    max_index = np.argmax(counts)
    majority_label = unique[max_index]
    return majority_label

def send_data_real_time(df, label, duration=15):
    for i in range(0, len(df), 15):  # Send 15 rows at a time
        if i + 15 > len(df):
            break  # Stop if there are fewer than 15 rows left
            
        # Adjust this slice if you need to drop a column (e.g., timestamp/ID)
        batch = df.iloc[i:i+15, :].values  
        batch = scaler.transform(batch)  # Apply scaling
        
        # Check if the number of features matches the model's expected number
        if batch.shape[1] != model.n_features_in_:
            print(f"Warning: Expected {model.n_features_in_} features, but got {batch.shape[1]} features.")
            continue  # Skip this batch if the number of features is incorrect
        
        predictions = model.predict(batch)  # Get predictions for the batch
        
        # Determine the majority class in the batch
        majority_label = get_majority_label(predictions)
        attack_type = attack_mapping.get(majority_label, "Unknown")
        
        print(f"[Dataset Label {label}] Predictions: {predictions}")
        print(f"-> Majority Vote: {majority_label} ({attack_type})")
        time.sleep(1)  # Wait for 1 second

# Run testing for each dataset sequentially
for df, label in datasets:
    send_data_real_time(df, label)


[Dataset Label 1] Predictions: [2 1 1 4 1 1 1 1 4 1 1 4 4 1 1]
-> Majority Vote: 1 (Attack Free)
[Dataset Label 1] Predictions: [4 4 1 1 1 1 4 1 1 4 1 1 1 1 1]
-> Majority Vote: 1 (Attack Free)
[Dataset Label 1] Predictions: [1 1 4 1 1 4 1 1 1 4 4 1 4 1 1]
-> Majority Vote: 1 (Attack Free)
[Dataset Label 1] Predictions: [1 4 4 1 1 1 1 3 1 1 4 1 1 1 1]
-> Majority Vote: 1 (Attack Free)
[Dataset Label 1] Predictions: [1 1 4 1 4 4 1 1 4 1 1 4 3 1 1]
-> Majority Vote: 1 (Attack Free)
[Dataset Label 1] Predictions: [1 1 1 4 1 1 1 2 4 4 1 1 3 1 4]
-> Majority Vote: 1 (Attack Free)
[Dataset Label 1] Predictions: [1 1 1 3 1 1 4 1 1 4 1 1 4 4 1]
-> Majority Vote: 1 (Attack Free)
[Dataset Label 1] Predictions: [4 4 1 3 1 1 1 1 1 1 1 1 4 1 1]
-> Majority Vote: 1 (Attack Free)
[Dataset Label 1] Predictions: [1 2 1 1 1 1 4 1 1 1 1 4 1 1 1]
-> Majority Vote: 1 (Attack Free)
[Dataset Label 1] Predictions: [1 4 4 4 2 1 1 2 1 1 1 1 1 1 4]
-> Majority Vote: 1 (Attack Free)
[Dataset Label 1] Predictions:

In [9]:
#lets same check for dt_model
import numpy as np
import pandas as pd
import time
import joblib  # For loading the trained model
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Load preprocessed datasets
attack_free = pd.read_csv('extracted feature/CAN_attack_dataset1/Attack_free new.csv')[0:500]
dos = pd.read_csv('extracted feature/CAN_attack_dataset1/DoS_Attack_new.csv')[0:200]
fuzzy = pd.read_csv('extracted feature/CAN_attack_dataset1/Fuzzy_Attack_New.csv')[0:300]
impersonation = pd.read_csv('extracted feature/CAN_attack_dataset1/Impersonation_Attack_New.csv')[0:400]

def preprocess_data(df):
    label_encoder = LabelEncoder()
    # Process all columns except (optionally) the first if it is not categorical
    for col in df.columns[1:]:
        df[col] = label_encoder.fit_transform(df[col])
    return df

# Apply preprocessing
attack_free = preprocess_data(attack_free)
dos = preprocess_data(dos)
fuzzy = preprocess_data(fuzzy)
impersonation = preprocess_data(impersonation)

# Combine datasets
datasets = [(attack_free, 1), (dos, 2), (fuzzy, 3), (impersonation, 4)]

# Feature Scaling: **Include all columns**
dataset_combined = np.vstack([df.values for df, _ in datasets])
scaler = StandardScaler()
scaler.fit(dataset_combined)

# Load the trained SVM model from the .pkl file
model = joblib.load("dt_model.pkl")  # This model expects 16 features

def send_data_real_time(df, label, duration=15):
    for i in range(0, len(df), 15):  # Send 15 rows at a time
        if i + 15 > len(df):
            break  # Stop if there are fewer than 15 rows left
        batch = df.iloc[i:i+15, :].values  # Use all columns (16 features)
        batch = scaler.transform(batch)     # Apply scaling
        
        # Check if the number of features matches the model's expected number
        if batch.shape[1] != model.n_features_in_:
            print(f"Warning: Expected {model.n_features_in_} features, but got {batch.shape[1]} features.")
            continue  # Skip this batch if the number of features is incorrect
        
        predictions = model.predict(batch)  # Use the loaded model
        print(f"[{label}] Predictions: {predictions}")
        time.sleep(1)  # Wait for 1 second

# Run testing for each dataset sequentially
for df, label in datasets:
    send_data_real_time(df, label)


[1] Predictions: [1 4 1 3 1 2 1 1 3 2 1 2 2 2 4]
[1] Predictions: [4 1 1 1 1 1 1 4 4 3 1 4 2 1 3]
[1] Predictions: [2 1 2 2 4 2 2 2 1 3 1 4 1 1 1]
[1] Predictions: [1 1 3 4 1 1 1 2 2 4 4 1 1 1 2]
[1] Predictions: [1 1 1 4 4 3 1 2 3 4 1 2 2 2 4]
[1] Predictions: [2 4 1 3 1 2 1 1 1 4 1 1 2 1 3]
[1] Predictions: [2 4 1 2 2 4 4 1 1 2 1 4 1 1 4]
[1] Predictions: [4 4 1 2 1 3 2 2 2 4 1 2 2 2 2]
[1] Predictions: [1 1 2 1 1 4 3 1 1 1 1 3 2 2 2]
[1] Predictions: [4 2 1 1 4 2 1 1 4 2 1 1 4 4 3]
[1] Predictions: [1 1 1 3 1 2 2 4 2 2 2 1 1 2 1]
[1] Predictions: [1 4 3 1 1 2 1 1 3 1 2 4 2 1 1]
[1] Predictions: [4 2 1 1 4 4 1 1 1 2 3 4 1 2 2]
[1] Predictions: [1 2 4 2 2 2 1 4 1 2 1 1 3 1 1]
[1] Predictions: [2 1 1 3 1 2 2 4 4 4 4 1 4 4 3]
[1] Predictions: [2 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1] Predictions: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1] Predictions: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1] Predictions: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1] Predictions: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1] Predictions: [1 

In [10]:
import numpy as np
import pandas as pd
import time
import joblib  # For loading the trained model
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Load preprocessed datasets (example with limited rows)
attack_free = pd.read_csv('extracted feature/CAN_attack_dataset1/Attack_free new.csv')[0:500]
dos = pd.read_csv('extracted feature/CAN_attack_dataset1/DoS_Attack_new.csv')[0:200]
fuzzy = pd.read_csv('extracted feature/CAN_attack_dataset1/Fuzzy_Attack_New.csv')[0:300]
impersonation = pd.read_csv('extracted feature/CAN_attack_dataset1/Impersonation_Attack_New.csv')[0:400]

def preprocess_data(df):
    label_encoder = LabelEncoder()
    for col in df.columns[1:]:  # Assuming first column is timestamp or ID
        df[col] = label_encoder.fit_transform(df[col])
    return df

# Apply preprocessing
attack_free = preprocess_data(attack_free)
dos = preprocess_data(dos)
fuzzy = preprocess_data(fuzzy)
impersonation = preprocess_data(impersonation)

# Combine datasets with a dataset label for reference
datasets = [(attack_free, 1), (dos, 2), (fuzzy, 3), (impersonation, 4)]

# Feature Scaling
scaler = StandardScaler()
# Note: Adjust here if you want to include or exclude the first column. 
# For example, if the first column is not a feature, use df.iloc[:, 1:]
dataset_combined = np.vstack([df.values for df, _ in datasets])
scaler.fit(dataset_combined)

# Load the trained SVM model from the .pkl file
model = joblib.load("dt_model.pkl")  # The model should expect the same number of features as scaler was fit on

# Mapping from numeric prediction to attack type
attack_mapping = {
    1: "Attack Free",
    2: "DoS",
    3: "Fuzzy",
    4: "Impersonation"
}

def get_majority_label(predictions):
    """
    Given an array of predictions, determine the majority label.
    """
    unique, counts = np.unique(predictions, return_counts=True)
    max_index = np.argmax(counts)
    majority_label = unique[max_index]
    return majority_label

def send_data_real_time(df, label, duration=15):
    for i in range(0, len(df), 15):  # Send 15 rows at a time
        if i + 15 > len(df):
            break  # Stop if there are fewer than 15 rows left
            
        # Adjust this slice if you need to drop a column (e.g., timestamp/ID)
        batch = df.iloc[i:i+15, :].values  
        batch = scaler.transform(batch)  # Apply scaling
        
        # Check if the number of features matches the model's expected number
        if batch.shape[1] != model.n_features_in_:
            print(f"Warning: Expected {model.n_features_in_} features, but got {batch.shape[1]} features.")
            continue  # Skip this batch if the number of features is incorrect
        
        predictions = model.predict(batch)  # Get predictions for the batch
        
        # Determine the majority class in the batch
        majority_label = get_majority_label(predictions)
        attack_type = attack_mapping.get(majority_label, "Unknown")
        
        print(f"[Dataset Label {label}] Predictions: {predictions}")
        print(f"-> Majority Vote: {majority_label} ({attack_type})")
        time.sleep(1)  # Wait for 1 second

# Run testing for each dataset sequentially
for df, label in datasets:
    send_data_real_time(df, label)


[Dataset Label 1] Predictions: [1 4 1 3 1 2 1 1 3 2 1 2 2 2 4]
-> Majority Vote: 1 (Attack Free)
[Dataset Label 1] Predictions: [4 1 1 1 1 1 1 4 4 3 1 4 2 1 3]
-> Majority Vote: 1 (Attack Free)
[Dataset Label 1] Predictions: [2 1 2 2 4 2 2 2 1 3 1 4 1 1 1]
-> Majority Vote: 1 (Attack Free)
[Dataset Label 1] Predictions: [1 1 3 4 1 1 1 2 2 4 4 1 1 1 2]
-> Majority Vote: 1 (Attack Free)
[Dataset Label 1] Predictions: [1 1 1 4 4 3 1 2 3 4 1 2 2 2 4]
-> Majority Vote: 1 (Attack Free)
[Dataset Label 1] Predictions: [2 4 1 3 1 2 1 1 1 4 1 1 2 1 3]
-> Majority Vote: 1 (Attack Free)
[Dataset Label 1] Predictions: [2 4 1 2 2 4 4 1 1 2 1 4 1 1 4]
-> Majority Vote: 1 (Attack Free)
[Dataset Label 1] Predictions: [4 4 1 2 1 3 2 2 2 4 1 2 2 2 2]
-> Majority Vote: 2 (DoS)
[Dataset Label 1] Predictions: [1 1 2 1 1 4 3 1 1 1 1 3 2 2 2]
-> Majority Vote: 1 (Attack Free)
[Dataset Label 1] Predictions: [4 2 1 1 4 2 1 1 4 2 1 1 4 4 3]
-> Majority Vote: 1 (Attack Free)
[Dataset Label 1] Predictions: [1 1 1 